In [5]:
# import library
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import time
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [6]:
# data_road
train_data = pd.read_csv("../data/train.tsv", delimiter='\t')
x, y = train_data.iloc[:, 3:21], train_data["LeagueIndex"]-1
x_train, y_train, x_test, y_test = train_test_split(x, y)

In [10]:
start_time = time.time()

dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

xgb_params = {'objective': 'multi:softmax', 'num_class': 8, 'eval_metric': 'accuracy'}

evals = [(dtrain, 'train'), (dtest, 'valid')]

xgb_model = xgb.train(xgb_params,
                      dtrain,
                      evals=evals)

print('elapsed_time:{}'.format(time.time()-start_time))

/Users/sugarl/.conda/envs/data_final/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


ValueError: Invalid shape: (425, 18) for label

In [ ]:
def k_fold_cv(x, y, model, k):
    cv = KFold(n_splits=k, random_state=0, shuffle=True)
    mse_list = []
    for train_index, test_index in cv.split(x):
        # get train and test data
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # fit model
        model.fit(x_train, y_train)
        # predict test data
        y_pred = model.predict(x_test)
        # loss
        mse = np.mean((y_pred - y_test)**2)
        mse_list.append(mse)
    print(f"MSE({k}FoldCV): {np.mean(mse_list)}")
    print(f"std: {np.std(mse_list)}")